## Mercedes-Benz 데이터셋

- 많은 공정 변수가 포함되어 있는 데이터셋을 통해서, 차원의 저주 문제를 해결해보고 여러가지 전처리 기법들을 적용해봅니다.

## 1. 라이브러리, 데이터 불러오기

In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터를 불러옵니다.
train = pd.read_csv("../input/mercedes-benz-greener-manufacturing/train.csv.zip")
test = pd.read_csv("../input/mercedes-benz-greener-manufacturing/test.csv.zip")

## 2. EDA

Exploratory Data Analysis

#### 찾은 특징들


1. 결측치 : 없음


2. dtype이 object인 column : X0 ~ X8까지 8개. (categorical feature)

> -> 어떻게 처리할지 고민해야함. (Ordinal Encoding VS One-Hot Encoding)

> -> categorical feature들은 종류 정보들이 알파벳으로 되어있으며(anomynized) 이 정보들 대비 target값의 차이가 있는지 확인.
(특별하게 관련 없음)

> -> binary feature들중에서 0만 가지고 있는 column들이 있음.

> -> 정보가 충분하지 않다고 판단(target value와의 관련성 0) 삭제.


3. target distribution
-> train data에 180을 넘는 데이터가 하나 있음. 이 데이터를 outlier라고 생각하고 제거.

In [ ]:
### outlier 찾아서 지우기!

plt.figure(figsize=(12, 8))
#sns.histplot(data=train, x="y")
#sns.boxplot(data=train, x="y", whis=3)
sns.boxplot(data=train, x="y")
## train에 y column이 161을 넘는 데이터를 제거하자
outliers = train.loc[train.y > 135] ## IQR*1.5
train = train.drop(index=outliers.index)
train

In [ ]:
# X0 ~ X8 : categorical variables --> 8개
cat_vars = train.columns[2:10]
cat_vars
# X10 ~ : binary variables
binary_vars = train.columns[10:]
binary_vars

In [ ]:
#train.X0.value_counts()
#plt.figure(figsize=(12, 8))
#sns.countplot(data=train, x="X3")

### Encoding Categorical features
# 1. Ordinal Encoding
for var in cat_vars:
    train[var] = pd.factorize(train[var])[0]

# 2. One-hot Encoding
#pd.get_dummies(data=train, columns=cat_vars)

In [ ]:
# binary variable들의 0과 1의 구성 비율을 확인합니다!

# 1. value_counts 함수를 이용한 방법
# temp = train.X10.value_counts(normalize=True).values * 100
# print(f"0의 비율: {temp[0]:.4f}%, 1의 비율: {temp[1]:.4f}%")

# 2. 1의 개수를 이용한 방법
all0_cols = [] # column에 있는 모든 데이터가 다 0인 케이스.

for col in binary_vars:
    one_percent = train[col].mean() * 100 # 1의 비율을 퍼센트로 표현
    if one_percent == 0.0: # 1의 개수가 0인 경우.
        all0_cols.append(col) ## 모든 값이 0인 column들을 추가합니다.
        
    print(f"[{col}]  0의 비율: {100 - one_percent:.2f}%\t1의 비율: {one_percent:.2f}%")

In [ ]:
train = train.drop(columns=all0_cols)
train

### 3. feature engineering

1. Correlation


2. VIF


3. PCA

In [ ]:
# 1. Correlation이 높은 변수들 찾기 --> 필요없는 column : ID, y
temp = train.drop(columns=["ID", "y"])
corr = temp.corr() # 363 x 363
threshold = 0.7

#display(corr.loc[corr.X1 > threshold])

# e.g. X1 ----> [X3, X6, X10]
#      X2 ----> [....]       
#      X3 ----> [X1, X6, X10, ..., ...] (skip)
#      X4 ----> [....]
high_correlated_cols = []

for col in temp.columns:
    if col in high_correlated_cols:
        continue
    temp_cols = list(corr.loc[(corr[col] > threshold) | (corr[col] < -threshold)].index[1:]) # 보고 있는 column을 제외하고 threshold보다 상관계수가 높은 column들을 리스트로 변환.
    high_correlated_cols = high_correlated_cols + temp_cols # 리스트 누적
    
high_correlated_cols = set(high_correlated_cols) # 중복 column들 제거
high_correlated_cols

In [ ]:
# 2. VIF가 10이 넘는 변수들 찾기
## VIF_i = 1 / 1 - R2_i

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_list = [variance_inflation_factor(temp, idx) for idx in range(len(temp.columns))]

vif_df = pd.DataFrame(index=temp.columns, data=vif_list, columns=["VIF"])
vif_df.sort_values(by="VIF", ascending=False) # VIF가 큰 순서대로 정렬해서 만든 DataFrame

## VIF가 가장 큰 (또는 10보다 큰 column중에 하나)를 골라서 제거 한 뒤에 다시 VIF 계산. (loop)
## Continue..

In [ ]:
# 3. PCA(Principal Component Analysis)를 이용하여 변수 크기 줄이기
from sklearn.decomposition import PCA

## PCA의 n_components -----> hyper-parameter!
# 1. n_components = 6 : 6차원으로 내려주세요.
# 2. n_components = 0.90 : 원본 데이터의 분포를 90% 보존하는 차원으로 내려주세요.

#pca = PCA(n_components=15)
pca = PCA(n_components=0.95)
X = pca.fit_transform(temp) # 주성분을 기준으로 데이터를 "저차원"으로 변환
pca_columns = [f"PC{i}" for i in range(X.shape[1])]
pca_df = pd.DataFrame(columns=pca_columns, data=X)
pca_df